# Building Data Lake for demographic analysis and prediction in the U.S.

## Data Engineering Capstone Project

### Project Summary
Main goal of the project is to building Data Lake for demographic analysis and prediction in the U.S.

There are several datasets with raw data:

* `I94 Immigration Data`;
* `U.S. City Demographic Data`;
* `Airport Codes`.

Project focuses on creating data model for the future analysis and building ETL-pipeline using Apache Spark. The whole ETL-pipeline processes raw data, cleanup, apply data quality checks and store to the analytical area in the Data Lake according to data model.
Resulting database allows analytics and data scientists to find and predict correlation between demographic situation in different cities/states and immigration to this locations.

The project follows the follow steps:

* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up11

In [1]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql import SparkSession

In [17]:
# Set pandas options
pd.set_option('display.width', 500)
pd.set_option('display.max_colwidth', 500)

In [4]:
# Create and configure Spark session. Add support for sas7bdat data format.
spark = (
    SparkSession
    .builder
    .config('spark.jars.packages','saurfang:spark-sas7bdat:2.0.0-s_2.11')
    .config('spark.sql.session.timeZone', 'UTC')
    .enableHiveSupport()
    .getOrCreate()
)

### Step 1: Scope the Project and Gather Data

#### Scope

We will build a database (in the analytical area of our data lake) for demographic analysis and predictions which depends on immegration data.
Our resulting database will have the following structure:

* `airports` – dimension table which contains data about U.S. international airports. This table filled from the raw dataset represented by [airport-codes_csv.csv](./airport-codes_csv.csv) data source file.
* `cities` – dimension table with U.S. cities that will interested us in the future analytics. This table filled from the raw dataset represented by [us-cities-demographics.json](./us-cities-demographics.json) datasource file. We want to analyze only data for the same cities to compare with aggregated data for the 2015 year.
* `calendar` – dimension table with year and month to have a possibility to monitor situation with month granularity. This table filled from the raw dataset represented by `I94 Immigration Data` SAS database.
* `immigration` – fact table which contains aggregated data grouped by year, month, and city. Each row of this table shows how much people (men, women and total) immgegrated to the particular U.S. city each month. This table filled from the raw dataset represented by `I94 Immigration Data` SAS database joined with `airports` and `cities` data.
* `demographics` – pre-aggregated materialized view which is good candidate to land in data mart and contains aggregated data grouped by year and city. It can be used for the fast analytics and reporting. This table filled in two steps:
  1. Filled from the `us-cities-demographics.csv` datasource file which already contains data for the 2015 year.
  2. Filled from the `immigration` fact table by additition to the data for the previous years which allows us to see growth of the population.

##### Important notes and assumptions

1. We assume that all immigrants from the `I94 Immigration Data` will come and stay in the U.S. Actually we could analyze their visa types to be more precises, but for the scope of this project we will not consider it.
2. We assume that all immigrants will stay for living in the city to which the airport belongs. In real life this may not be the case, but we will not consider it.
3. Actually `demographics` table will contains data only for the 2015 and 2016 years because the `I94 Immigration Data` dataset contains data only for the 2016 year but our pipeliens will work for datasets with following years too.

##### Tools

We will use [Apache Spark](https://spark.apache.org/) to build our pipeline. Also we will use Data Lake idea to build our resulting tables and store our data in the HDFS or object storage like S3. We will not create staging area because in the Data Lake approache we can just load raw datasets with Spark, make all necessary transformations, cleanup the data, apply data quality checks and land it in the analytical area.

#### Describe and Gather Data

We will use several data sources to achieve our goal.

1. We will use `U.S. City Demographic Data` grouped by city for 2015 year provided by [U.S. Census Bureau](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/information/).
  * Data format: [JSON](https://en.wikipedia.org/wiki/JSON). You may see both JSON and CSV files with the same dataset in the repository, but we choose JSON format because we want to use different formats for each dataset.
  * Data source: [us-cities-demographics.json](./us-cities-demographics.json)
2. We will extract U.S. immigration data grouped by year, month and city from `I94 Immigration Data` provided by [US National Tourism and Trade Office](https://travel.trade.gov/research/reports/i94/historical/2016.html).
  * Data format: [SAS7BDAT](https://cran.r-project.org/web/packages/sas7bdat/vignettes/sas7bdat.pdf)
  * Data format description: SAS labels and possible values and errors described in the [I94_SAS_Labels_Descriptions.SAS](./I94_SAS_Labels_Descriptions.SAS) file.
  * Data source: [i94_apr16_sub.sas7bdat](../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat). Actually there are bunch of files partitioned by month.
3. Also we will use [Airport Code Table](https://datahub.io/core/airport-codes#data).
  * Data format: [CSV](https://en.wikipedia.org/wiki/Comma-separated_values)
  * Data source: [airport-codes_csv.csv](./airport-codes_csv.csv)

#### U.S. City Demographic Data

Extract raw data from the data source and display few rows.

As you may see in the example below demogrphic data stored in the `fields` column. There are several fields which we have to extract to achieve our goal:

* city;
* state;
* state_code;
* male_population;
* female_population;
* total_population;

In [16]:
# Load raw data from the U.S. City Demographic Data and display first 5 rows
demographics_source_file_name = './us-cities-demographics.json'
demographics_demo_df = pd.read_json(demographics_source_file_name)
demographics_demo_df.head()

,datasetid,fields,record_timestamp,recordid
0,us-cities-demographics,"{'count': 2177650, 'city': 'Los Angeles', 'number_of_veterans': 85417, 'male_population': 1958998, 'foreign_born': 1485425, 'average_household_size': 2.86, 'median_age': 35.0, 'state': 'California', 'race': 'White', 'total_population': 3971896, 'state_code': 'CA', 'female_population': 2012898}",1970-01-01T03:00:00+03:00,7da42fda61238faccac3d43954a8f621a3a51194
1,us-cities-demographics,"{'count': 124270, 'city': 'Metairie', 'number_of_veterans': 7187, 'male_population': 69515, 'foreign_born': 19871, 'average_household_size': 2.39, 'median_age': 41.6, 'state': 'Louisiana', 'race': 'White', 'total_population': 146458, 'state_code': 'LA', 'female_population': 76943}",1970-01-01T03:00:00+03:00,f176fd28e69ee0c152db080858781e769840c6cc
2,us-cities-demographics,"{'count': 80781, 'city': 'Boca Raton', 'number_of_veterans': 4367, 'male_population': 44760, 'foreign_born': 21117, 'average_household_size': 2.22, 'median_age': 47.3, 'state': 'Florida', 'race': 'White', 'total_population': 93226, 'state_code': 'FL', 'female_population': 48466}",1970-01-01T03:00:00+03:00,ed4a94bf11b553ed8ebd93b7c24b0e8f326dadb0
3,us-cities-demographics,"{'count': 2566, 'city': 'Quincy', 'number_of_veterans': 4147, 'male_population': 44129, 'foreign_born': 32935, 'average_household_size': 2.39, 'median_age': 41.0, 'state': 'Massachusetts', 'race': 'Hispanic or Latino', 'total_population': 93629, 'state_code': 'MA', 'female_population': 49500}",1970-01-01T03:00:00+03:00,a4cd5f6782c79aa0348652718ca179ca390ce086
4,us-cities-demographics,"{'count': 16845, 'city': 'Union City', 'number_of_veterans': 1440, 'male_population': 38599, 'foreign_born': 32752, 'average_household_size': 3.46, 'median_age': 38.5, 'state': 'California', 'race': 'White', 'total_population': 74510, 'state_code': 'CA', 'female_population': 35911}",1970-01-01T03:00:00+03:00,b3b36950de605af92a78168f5127dff485a59b0b


#### I94 Immigration Data

Extract raw data from the data source and display few rows.

There are several fields which we have to extract to achieve our goal:

* `cicid` – unique row identifier;
* `i94yr` – year in 4 digit format;
* `i94mon` – month index (Jan = 1, Feb = 2, Mar = 3, etc.);
* `i94port` – airport code in 3 letters format described in the SAS file description;
* `i94mode` – border crossing method (Air = 1, Sea = 2, Land = 3, Not reported = 9);
* `i94addr` – U.S. state of arrival;
* `gender` – male (M) or female (F).

In [ ]:
# Load raw data from the I94 Immigration Data and display first 5 rows
immigration_source_file_name = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
immigration_demo_df = pd.read_sas(immigration_source_file_name, 'sas7bdat', encoding='ISO-8859-1')
immigration_demo_df.head()

In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [11]:
#write to parquet
df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness

Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

In [ ]:
#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.